<a href="https://colab.research.google.com/github/okay2528/Fastai_Lesson_Project/blob/master/NYC_TLC_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# here's how I modify the notebook and run on Colab fine
from google.colab import auth
auth.authenticate_user()  # login

import os
os.environ['PROJECT_ID'] = 'quiet-being-241109'

# !pip install datalab
import google.datalab.bigquery as bq

# # then just query
# sql = 'SELECT 1'
# bq.Query(sql).execute().result().to_dataframe()

import pandas as pd

# Load data

In [ ]:
## Review data with sample 1000 records
# Create and run a SQL query for the taxi_data from 2021 and 2022
# sql_code = '(SELECT * FROM `bigquery-public-data.new_york_taxi_trips.tlc_yellow_trips_2021` LIMIT 1000) \
#                    union all (SELECT * FROM `bigquery-public-data.new_york_taxi_trips.tlc_yellow_trips_2022` LIMIT 1000)'
sql_code = '(SELECT pickup_datetime,	dropoff_datetime, trip_distance, fare_amount, total_amount, pickup_location_id,	dropoff_location_id \
                      from `bigquery-public-data.new_york_taxi_trips.tlc_yellow_trips_2021` \
                      where fare_amount > 0\
                      and trip_distance > 0\
                      and total_amount > 0\
                      and passenger_count >0)'
query = bq.Query(sql_code)
output_options = bq.QueryOutput.table(use_cache=False)
result = query.execute(output_options=output_options).result()
 
# Convert to DataFrame
df = result.to_dataframe()
df.shape[0]

In [ ]:
df.describe()

In [ ]:
df.head()

# Prepare data

In [ ]:
# Calculate trip_duration in minutes
df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'])
df['dropoff_datetime'] = pd.to_datetime(df['dropoff_datetime'])
df['trip_duration'] = (df['dropoff_datetime'] - df['pickup_datetime']).dt.total_seconds()/60

In [ ]:
# Visualise the columns to be used for simple intuitive model
pd.plotting.scatter_matrix(frame = df[['fare_amount', 'trip_duration']], figsize=(7,7))


In [ ]:
# # Remove the outliers
# df_filtered = df[(df['trip_distance'] >= 2) & (df['trip_distance'] <= 50) &\
#                  (df['fare_amount'] >= 3) & (df['fare_amount'] <=300) & \
#                  (df['trip_duration'] <= 200) & (df['trip_duration'] > 1)].copy()

# # Visualise the columns to be used for simple intuitive model
# pd.plotting.scatter_matrix(frame = df_filtered[['fare_amount', 'trip_distance', 'trip_duration']], figsize=(7,7));

In [ ]:
# Write to csv
df.to_csv('data.csv')